In [120]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb

In [121]:
import subprocess as sub

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information.

## TODO:

- Add some sort of typing system. var r:int = 10
- Inject block info

In [199]:
code = """
def init():
    r = 0
    s = 2
    h = 'hello how are you?'

r += 5
time = _tx["time"] 
"""

In [270]:
SC = 'sc.py'

PRINT_VARS = ['local = set(locals())',
              'local_vars = {}',
              'for n in local:',
              '    if not n.startswith(\'_\'):',
              '         if \'str\' in str(type(eval(n))): print(n, f\'\"{eval(n)}\"\', \'str\')',
              '         else: print(n, eval(n), \'int\')']

In [271]:
MEMORY = {}

In [272]:
def inject(tx, code): 
    tx = tx.json().replace('true', 'True')
    return '_tx = '+tx+'\n'+code

In [273]:
def save(code):
    with open(SC, 'w+') as f: f.write(code)

In [274]:
def run():
    r = sub.check_output(f'python3 {SC}', shell=True)
    return r.decode().split('\n')[:-1]

In [275]:
def store(r):
    for a in r:
        if 'str' in a:
            b = a.split('"')
            MEMORY[b[0].strip()] = [b[1], b[2].strip()]
        else: 
            b = a.split(' ')
            MEMORY[b[0]] = [b[1], b[2]]

In [276]:
def deployment(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].strip())
                else                  : break
    init.extend(PRINT_VARS)
    return '\n'.join(l for l in init)

In [277]:
def deploy(code, tx):
    dc = deployment(code)
    ic = inject(tx, dc)
    save(ic)
    r  = run()
    store(r)

deploy(code, tx1); MEMORY

{'r': ['0', 'int'], 'h': ['hello how are you?', 'str'], 's': ['2', 'int']}

In [171]:
def execution(code):
    c = []
    for k,v in MEMORY.items():
        if v[1] == 'str': c.append(f'{k} = "{v[0]}"')
        else            : c.append(f'{k} = {v[0]}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: c.append(l)
    c.extend(PRINT_VARS)
    return '\n'.join(l for l in c)

In [172]:
def exe(code, tx):
    ic = inject(tx, code)
    ec = execution(ic)
    save(ec)
    r  = run()
    print(r)
    store(r)
    
exe(code, tx1)

['s 2 int', 'r 5 int', 'time Mon Apr 12 02:23:37 2021 str', 'h hellohowareyou? str']


In [173]:
MEMORY

{'r': ['5', 'int'],
 'h': ['hellohowareyou?', 'str'],
 's': ['2', 'int'],
 'time': ['Mon', 'Apr']}